In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
import numpy as np
import pandas as pd
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator, GetAtomPairGenerator
from rdkit.Chem import MACCSkeys

import umap
import base64
from io import BytesIO

In [3]:
df = pd.read_csv('small_molecule.csv')

In [4]:
df

,SMILES,ChEBI ID,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,UMAP_1,UMAP_2
0,**,['17792'],"['P00502', 'P04904', 'P08263', 'P0CG30', 'P137...","['GSTA1_RAT', 'GSTA3_RAT', 'GSTA1_HUMAN', 'GST...",['Glutathione S-transferase alpha-1 (EC 2.5.1....,"['Gsta1', 'Gsta3 Gstyc1', 'GSTA1', 'GSTT2B GST...","['Rattus norvegicus (Rat)', 'Rattus norvegicus...","[222, 221, 222, 244, 223, 221, 222, 222]",['MSGKPVLHYFNARGRMECIRWLLAAAGVEFDEKFIQSPEDLEKL...,6.944330,15.337144
1,C(SCCNC(CCNC(=O)[C@@H](C(COP([O-])(=O)[O-])(C)...,['132021'],"['P0C024', 'Q99P30']","['NUDT7_HUMAN', 'NUDT7_MOUSE']",['Peroxisomal coenzyme A diphosphatase NUDT7 (...,"['NUDT7', 'Nudt7']","['Homo sapiens (Human)', 'Mus musculus (Mouse)']","[238, 236]",['MSRLGLPEEPVRNSLLDDAKARLRKYDIGGKYSHLPYNKYSVLL...,1.092276,15.910781
2,C12(C=CC(C=C1CCC3C2CCC4(C3CCC4*)C)=O)C,['77166'],['P71864'],['3O1D_MYCTU'],['3-oxosteroid 1-dehydrogenase (EC 1.3.99.4) (...,['kstD Rv3537'],['Mycobacterium tuberculosis (strain ATCC 2561...,[563],['MTVQEFDVVVVGSGAAGMVAALVAAHRGLSTVVVEKAPHYGGST...,7.832362,7.765767
3,C12(CCCCC1CCC3C2C(CC4(C3CCC4*)C)=O)C,['47787'],"['A0A096MXN2', 'A0A218UG05', 'A0A2J8V6J2', 'A0...","['A0A096MXN2_PAPAN', 'A0A218UG05_9PASE', 'A0A2...",['11-beta-hydroxysteroid dehydrogenase 1 (EC 1...,"['HSD11B1', 'HSD11B1_1 RLOC_00001311', 'HSD11B...","['Papio anubis (Olive baboon)', 'Lonchura stri...","[292, 363, 292, 291, 292, 292, 292, 292, 292, ...",['MAFMKKYLLPILGLFMAYYYYSAYEEFRPEMLQGKKVIVTGASK...,6.665026,15.332088
4,C12(CCCCC1CCC3C2[C@H](CC4(C3CCC4*)C)O)C,['35346'],"['A0A096MXN2', 'A0A218UG05', 'A0A2J8V6J2', 'A0...","['A0A096MXN2_PAPAN', 'A0A218UG05_9PASE', 'A0A2...",['11-beta-hydroxysteroid dehydrogenase 1 (EC 1...,"['HSD11B1', 'HSD11B1_1 RLOC_00001311', 'HSD11B...","['Papio anubis (Olive baboon)', 'Lonchura stri...","[292, 363, 292, 291, 292, 292, 292, 292, 292, ...",['MAFMKKYLLPILGLFMAYYYYSAYEEFRPEMLQGKKVIVTGASK...,6.637052,15.422254
...,...,...,...,...,...,...,...,...,...,...,...
546,[H][C@]12CC[C@]3([H])[C@]([H])(C[C@@H](O)[C@]4...,['71002'],['C8WLM1'],['CGR2_EGGLE'],['Digoxin reductase (EC 1.3.2.-) (Cardenolide ...,['cgr2 Elen_2529'],['Eggerthella lenta (strain ATCC 25559 / DSM 2...,[560],['MEYGKCRGIERGMGRRDFLKAATLLGATAAGAGMLAGCAPKSAS...,5.709255,13.093595
547,[H][C@]12C[C@@H](O)[C@@]3([H])[C@]4([H])CC[C@]...,['2288'],"['P31210', 'P51857', 'Q8VCX1', 'Q9TV64']","['AK1D1_RAT', 'AK1D1_HUMAN', 'AK1D1_MOUSE', 'A...",['Aldo-keto reductase family 1 member D1 (EC 1...,"['Akr1d1', 'AKR1D1 SRD5B1', 'Akr1d1', 'AKR1D1']","['Rattus norvegicus (Rat)', 'Homo sapiens (Hum...","[326, 326, 325, 326]",['MNLSTANHHIPLNDGNSIPIIGLGTYSDPRPVPGKTFIAVKTAI...,1.438020,14.527494
548,[H][C@]12C[C@@H](O)[C@]3([H])[C@]([H])(CC[C@]4...,['2290'],"['P31210', 'P51857', 'Q8VCX1', 'Q9TV64']","['AK1D1_RAT', 'AK1D1_HUMAN', 'AK1D1_MOUSE', 'A...",['Aldo-keto reductase family 1 member D1 (EC 1...,"['Akr1d1', 'AKR1D1 SRD5B1', 'Akr1d1', 'AKR1D1']","['Rattus norvegicus (Rat)', 'Homo sapiens (Hum...","[326, 326, 325, 326]",['MNLSTANHHIPLNDGNSIPIIGLGTYSDPRPVPGKTFIAVKTAI...,1.553790,14.263593
549,[H][C@]12C[C@@]3([H])[C@]4([H])CC=C5C[C@H](CC[...,['2938'],['Q38786'],['AVCO1_AVESA'],['Avenacosidase 1 (EC 3.2.1.188) (26-desgluco-...,['P60A GLU1'],['Avena sativa (Oat)'],[574],['MALLCSALSNSTHPSFRSHIGANSENLWHLSADPAQKSKRRCNL...,2.996281,5.881175


In [5]:
# Initialize new generators
morgan_gen = GetMorganGenerator(radius=2, fpSize=1024)
atom_pair_gen = GetAtomPairGenerator(fpSize=1024)

# Function to compute molecular fingerprints
def compute_fingerprints(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None, None, None

    # Morgan Fingerprint
    morgan_fp = morgan_gen.GetFingerprint(mol)
    morgan_fp_array = np.array(morgan_fp)

    # MACCS Keys Fingerprint
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    maccs_fp_array = np.array(maccs_fp)

    # Atom-Pair Fingerprint
    atom_pair_fp = atom_pair_gen.GetFingerprint(mol)
    atom_pair_fp_array = np.array(atom_pair_fp)

    return morgan_fp_array, maccs_fp_array, atom_pair_fp_array

# Compute fingerprints for each molecule
df["Morgan_FP"], df["MACCS_FP"], df["AtomPair_FP"] = zip(*df["SMILES"].apply(compute_fingerprints))

# Save or Display the updated dataframe
# df.to_csv("steroid_fingerprints.csv", index=False)
df.head()

,SMILES,ChEBI ID,Entry,Entry Name,Protein names,Gene Names,Organism,Length,Sequence,UMAP_1,UMAP_2,Morgan_FP,MACCS_FP,AtomPair_FP
0,**,['17792'],"['P00502', 'P04904', 'P08263', 'P0CG30', 'P137...","['GSTA1_RAT', 'GSTA3_RAT', 'GSTA1_HUMAN', 'GST...",['Glutathione S-transferase alpha-1 (EC 2.5.1....,"['Gsta1', 'Gsta3 Gstyc1', 'GSTA1', 'GSTT2B GST...","['Rattus norvegicus (Rat)', 'Rattus norvegicus...","[222, 221, 222, 244, 223, 221, 222, 222]",['MSGKPVLHYFNARGRMECIRWLLAAAGVEFDEKFIQSPEDLEKL...,6.944330,15.337144,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,C(SCCNC(CCNC(=O)[C@@H](C(COP([O-])(=O)[O-])(C)...,['132021'],"['P0C024', 'Q99P30']","['NUDT7_HUMAN', 'NUDT7_MOUSE']",['Peroxisomal coenzyme A diphosphatase NUDT7 (...,"['NUDT7', 'Nudt7']","['Homo sapiens (Human)', 'Mus musculus (Mouse)']","[238, 236]",['MSRLGLPEEPVRNSLLDDAKARLRKYDIGGKYSHLPYNKYSVLL...,1.092276,15.910781,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ..."
2,C12(C=CC(C=C1CCC3C2CCC4(C3CCC4*)C)=O)C,['77166'],['P71864'],['3O1D_MYCTU'],['3-oxosteroid 1-dehydrogenase (EC 1.3.99.4) (...,['kstD Rv3537'],['Mycobacterium tuberculosis (strain ATCC 2561...,[563],['MTVQEFDVVVVGSGAAGMVAALVAAHRGLSTVVVEKAPHYGGST...,7.832362,7.765767,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,C12(CCCCC1CCC3C2C(CC4(C3CCC4*)C)=O)C,['47787'],"['A0A096MXN2', 'A0A218UG05', 'A0A2J8V6J2', 'A0...","['A0A096MXN2_PAPAN', 'A0A218UG05_9PASE', 'A0A2...",['11-beta-hydroxysteroid dehydrogenase 1 (EC 1...,"['HSD11B1', 'HSD11B1_1 RLOC_00001311', 'HSD11B...","['Papio anubis (Olive baboon)', 'Lonchura stri...","[292, 363, 292, 291, 292, 292, 292, 292, 292, ...",['MAFMKKYLLPILGLFMAYYYYSAYEEFRPEMLQGKKVIVTGASK...,6.665026,15.332088,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,C12(CCCCC1CCC3C2[C@H](CC4(C3CCC4*)C)O)C,['35346'],"['A0A096MXN2', 'A0A218UG05', 'A0A2J8V6J2', 'A0...","['A0A096MXN2_PAPAN', 'A0A218UG05_9PASE', 'A0A2...",['11-beta-hydroxysteroid dehydrogenase 1 (EC 1...,"['HSD11B1', 'HSD11B1_1 RLOC_00001311', 'HSD11B...","['Papio anubis (Olive baboon)', 'Lonchura stri...","[292, 363, 292, 291, 292, 292, 292, 292, 292, ...",['MAFMKKYLLPILGLFMAYYYYSAYEEFRPEMLQGKKVIVTGASK...,6.637052,15.422254,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
# Convert fingerprint list to NumPy array
fingerprints = df["Morgan_FP"].tolist()  # Using Morgan Fingerprints
fingerprints_np = np.array(fingerprints)
umap_model = umap.UMAP(n_neighbors=5, min_dist=0.3, metric="euclidean")
embedding = umap_model.fit_transform(fingerprints_np)

# Add UMAP coordinates to the dataframe
df["UMAP_1"] = embedding[:, 0]
df["UMAP_2"] = embedding[:, 1]

AttributeError: module 'umap' has no attribute 'UMAP'

In [12]:
df

,Compounds,SMILES,logP,Molecular Weight,Morgan_FP,MACCS_FP,AtomPair_FP,UMAP_1,UMAP_2
0,Compound 1,[H][C@]12CC[C@]3([H])[C@]([H])(CC[C@]4(C)[C@@H...,3.7509,292.463,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-5.742396,11.522367
1,Compound 2,[H][C@@]12CCC3=CC(=O)CC[C@]3(C)[C@@]1([H])CC[C...,4.5153,316.485,"[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-5.823915,5.520349
2,Compound 3,[H][C@@]12CC[C@]([H])([C@H](C)CC[C@H](O)C(C)C)...,6.2796,400.647,"[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, ...",5.748560,-3.768179
3,Compound 4,*C1CCC2C3CCC4CCCCC4(C)C3[C@@H](O)CC12C,4.7278,275.456,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-4.894306,12.032340
4,Compound 5,CC(C)CC[C@H](O)C(C)C1CCC2C3CCC4CCCCC4(C)C3CCC21C,7.4686,388.680,"[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",-4.974775,2.083085
...,...,...,...,...,...,...,...,...,...
795,Compound 796,[H][C@]12CC[C@]3([H])[C@]([H])(CC[C@@]4(C)[C@@...,3.6180,578.787,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, ...",4.534284,3.701876
796,Compound 797,[H][C@@]12CCC3C[C@H](O)CC[C@]3(C)[C@@]1([H])CC...,3.7509,292.463,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-5.786797,11.466669
797,Compound 798,[H][C@@]12CC[C@]([H])(C(=O)C=O)[C@@]1(C)CC(=O)...,3.0815,342.435,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-3.951260,6.038994
798,Compound 799,[H][C@@]12CC[C@]3([H])[C@]([H])(CC[C@]4(C)C(=O...,3.3463,302.414,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",-6.874006,9.479789


In [7]:
df.to_csv('umap_smiles.csv')